In [1]:
import pandas as pd
import numpy as np
import sys
import logging
import emoji 

sys.path.append("src")
from song_search import find_nearest_song_annoy

NameError: name 'embed' is not defined

In [4]:
lyrics = pd.read_json('../data/sample_data/top_10_spotify_translated.jsonv')
emojis = lyrics['translated_lyrics'].values

def get_emoji_name(emoji_char):
    emoji_info = emoji.demojize(emoji_char)
    emoji_name = emoji_info.strip(':').replace('\u200d', '')
    if len(emoji_name) > 0:
        return emoji_name
    return ''

# get a song name by an emoji sequence
def avg_pos(data, n=10):
    matched = []
    for i in range(n):
        match = lyrics['song_name'].iloc[i]

        input_emoji = data[i]

        # emoji lib
        input_emoji_names = [get_emoji_name(emoji_char) for emoji_char in input_emoji \
            if len(get_emoji_name(emoji_char)) > 0]
        concatenated_names = '.'.join(input_emoji_names)

        guess = find_nearest_song_annoy(concatenated_names, 10000) + [match]
        print(guess.index(match), concatenated_names)
        matched.append(guess.index(match))

    return np.mean(matched)

def test_accuracy_lyrics2emoji_search():
    accuracy = avg_pos(emojis, len(emojis))
    logging.critical(f'mean searched pos: {accuracy}')



In [5]:
test_accuracy_lyrics2emoji_search()

21 brick.thinking_face.building_construction.brick.hammer
74 automobile.locked.TOP_arrow.ladder.folded_hands
193 thinking_face.boy.shield.girl
86 musical_note.drum.loudspeaker
729 musical_note.microphone.crown
5 left_arrow


CRITICAL:root:mean searched pos: 234.7


513 thinking_face.cityscape.person_shrugging.male_sign.cityscape
0 woman.school.school.backhand_index_pointing_left.red_question_mark
641 people_with_bunny_ears.female_sign.person_running.female_sign
85 ear.shushing_face
